In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [ ]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, updatepivots!, addglobalpivots1sitesweep!, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [ ]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test

In [ ]:
pivotsearch = :full
seed = 123
strictlynested = false

Random.seed!(seed)

R = 20
abstol = 1e-4
δ = 10.0 / 2^R # Peaks are wider than 1/2^R.
grid = QD.DiscretizedGrid{1}(R, (0.0,), (1.0,))

rindex = [rand(1:2, R) for _ in 1:10]

f(bitlist) = any(bitlist .> 2) ? 0.0 : fx(QD.quantics_to_origcoord(grid, bitlist)[1])
rpoint = Float64[QD.quantics_to_origcoord(grid, r)[1] for r in rindex]

function fx(x)
    res = exp(-10 * x) + 1.0/(x+1.0)
    for r in rpoint
        res += abs(x - r) < δ ? 2 * abstol : 0.0
    end
    res
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
    nsearchglobalpivot=0
)

In [ ]:
TCI.addglobalpivots2sitesweep!(tci, f, rindex; fillsitetensors=false, verbosity=4)

In [ ]:
for b in 1:length(tci)-1
    @show length(tci.Iset[b+1]), length(tci.Jset[b])
end

In [ ]:
TCI.fillsitetensors!(tci, f)

In [ ]:
length(tci.globalpivots)

In [ ]:
tt = TCI.TensorTrain(tci)
@show TCI.linkdims(tt)
TCI.compress!(tt, :SVD; tolerance=1e-14)
@show TCI.linkdims(tt)

In [ ]:
@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex]) .> abstol) == 0

rindex_test = [rand(1:2, R) for _ in 1:100]

@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex_test]) .> abstol) == 0